# Pose Error Project
### (Transformer-Encoder)

In [1]:
import logging

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Loading packages ...")
import os
import sys
import time
import pickle
import json

# 3rd party packages

#from tqdm import tqdm
# since we are using it in jupyter notebook
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# Project modules
from options import Options
from running import setup, pipeline_factory, validate, check_progress, NEG_METRICS
from utils import utils
from datasets.data import data_factory, Normalizer
from datasets.datasplit import split_dataset
from models.ts_transformer import model_factory
from models.loss import get_loss_module
from optimizers import get_optimizer

import parser


2023-05-10 17:07:31,527 | INFO : Loading packages ...
2023-05-10 17:07:32,800 | INFO : Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-05-10 17:07:32,801 | INFO : NumExpr defaulting to 8 threads.


# Setup Configurations

In [2]:
# Setting 1 - Single Stage
# Training From Scratch
text = "--output_dir ../experiments/ --comment 'pm25_from_Scratch' \
        --name pm25_fromScratch_Regression --records_file Regression_records.xls \
        --data_dir ../data/regression/BeijingPM25Quality/ --data_class tsra \
        --epochs 100 --lr 0.001 --optimizer RAdam \
        --pos_encoding learnable --task regression"

In [3]:
# Setting 2 - Two Stages
# Pretrain
'''
text = "--output_dir ../experiments/ --comment 'pretraining_through_imputation' \
        --name pm25_pretrained --records_file Imputation_records.xls \
        --data_dir ../data/regression/BeijingPM25Quality/ --data_class tsra \
        --pattern TRAIN --val_ratio 0.2 --epochs 20 --lr 0.001 --optimizer RAdam \
        --batch_size 32 --pos_encoding learnable --d_model 128"
'''

# Finetune
'''
text = "--output_dir ../experiments --comment 'finetune_for_regression' \
        --name BeijingPM25Quality_finetuned --records_file Regression_records.xls \
        --data_dir ../data/regression/BeijingPM25Quality/ --data_class tsra \
        --pattern TRAIN --val_pattern TEST  --epochs 100 --lr 0.001 --optimizer RAdam \
        --pos_encoding learnable --d_model 128 \
        --load_model ../experiments/pm25_pretrained_2023-05-04_11-01-18_AhP/checkpoints/model_best.pth \
        --task regression --change_output --batch_size 128"
'''

'\ntext = "--output_dir ../experiments --comment \'finetune_for_regression\'         --name BeijingPM25Quality_finetuned --records_file Regression_records.xls         --data_dir ../data/regression/BeijingPM25Quality/ --data_class tsra         --pattern TRAIN --val_pattern TEST  --epochs 100 --lr 0.001 --optimizer RAdam         --pos_encoding learnable --d_model 128         --load_model ../experiments/pm25_pretrained_2023-05-04_11-01-18_AhP/checkpoints/model_best.pth         --task regression --change_output --batch_size 128"\n'

In [4]:
# Process the setting string
# Generate the config variable
input_text = text.split()
args = Options().parse(input_text)
config = setup(args)

2023-05-10 17:07:32,939 | INFO : Stored configuration file in '../experiments/pm25_fromScratch_Regression_2023-05-10_17-07-32_kDk'


In [5]:
config

{'config_filepath': None,
 'output_dir': '../experiments/pm25_fromScratch_Regression_2023-05-10_17-07-32_kDk',
 'data_dir': '../data/regression/BeijingPM25Quality/',
 'load_model': None,
 'resume': False,
 'change_output': False,
 'save_all': False,
 'experiment_name': 'pm25_fromScratch_Regression',
 'comment': "'pm25_from_Scratch'",
 'no_timestamp': False,
 'records_file': 'Regression_records.xls',
 'console': False,
 'print_interval': 1,
 'gpu': '0',
 'n_proc': -1,
 'num_workers': 0,
 'seed': None,
 'limit_size': None,
 'test_only': None,
 'data_class': 'tsra',
 'labels': None,
 'test_from': None,
 'test_ratio': 0,
 'val_ratio': 0.2,
 'pattern': None,
 'val_pattern': None,
 'test_pattern': None,
 'normalization': 'standardization',
 'norm_from': None,
 'subsample_factor': None,
 'task': 'regression',
 'masking_ratio': 0.15,
 'mean_mask_length': 3,
 'mask_mode': 'separate',
 'mask_distribution': 'geometric',
 'exclude_feats': None,
 'mask_feats': [0, 1],
 'start_hint': 0.0,
 'end_hint

# Setup Logger

In [6]:
# Add file logging besides stdout
file_handler = logging.FileHandler(os.path.join(config['output_dir'], 'output.log'))
logger.addHandler(file_handler)

logger.info('Running:\n{}\n'.format(' '.join(sys.argv)))  # command used to run

2023-05-10 17:07:32,949 | INFO : Running:
/home/tianyi/anaconda3/envs/transformer/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/tianyi/.local/share/jupyter/runtime/kernel-efb3a0ae-2589-4ac8-8875-e056a8afd634.json



# Setup Training Device

In [7]:
if config['seed'] is not None:
    torch.manual_seed(config['seed'])

device = torch.device('cuda' if (torch.cuda.is_available() and config['gpu'] != '-1') else 'cpu')
logger.info("Using device: {}".format(device))
if device == 'cuda':
    logger.info("Device index: {}".format(torch.cuda.current_device()))

2023-05-10 17:07:32,990 | INFO : Using device: cuda


# Prepare Data

In [8]:
 # Build data
logger.info("Loading and preprocessing data ...")
data_class = data_factory[config['data_class']]
my_data = data_class(config['data_dir'], 
                     pattern=config['pattern'], 
                     n_proc=config['n_proc'], 
                     limit_size=config['limit_size'], 
                     config=config)
feat_dim = my_data.feature_df.shape[1]  # dimensionality of data features
if config['task'] == 'classification':
    validation_method = 'StratifiedShuffleSplit'
    labels = my_data.labels_df.values.flatten()
else:
    validation_method = 'ShuffleSplit'
    labels = None

2023-05-10 17:07:32,995 | INFO : Loading and preprocessing data ...
5072it [00:10, 488.12it/s]


In [9]:
feat_dim = my_data.feature_df.shape[1]
feat_dim

9

In [10]:
my_data.feature_df
#my_data.all_IDs[0:50]

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8
0,47.0,123.0,5300.0,13.0,-2.5,1024.5,-8.2,0.0,1.1
0,66.0,119.0,5400.0,16.0,-3.5,1023.8,-7.3,0.0,1.0
0,64.0,115.0,4300.0,12.0,-4.7,1023.7,-7.3,0.0,0.8
0,56.0,111.0,4100.0,9.0,-3.6,1023.1,-7.7,0.0,1.5
0,46.0,107.0,3600.0,7.0,-5.1,1022.3,-7.7,0.0,1.0
...,...,...,...,...,...,...,...,...,...
5047,3.0,24.0,400.0,72.0,12.5,1013.5,-16.2,0.0,2.4
5047,3.0,41.0,500.0,50.0,11.6,1013.6,-15.1,0.0,0.9
5047,4.0,38.0,500.0,54.0,10.8,1014.2,-13.3,0.0,1.1
5047,4.0,30.0,400.0,59.0,10.5,1014.4,-12.9,0.0,1.2


In [11]:
type(my_data.labels_df)

pandas.core.frame.DataFrame

In [12]:
286032.0/11918.0

24.0

In [13]:
#from datasets import utils
#x, y = utils.load_from_tsfile_to_dataframe("../data/regression/BeijingPM25Quality/BeijingPM25Quality_TRAIN.ts")
#y.shape
#x

In [14]:
total_epoch_time = 0
total_eval_time = 0

total_start_time = time.time()

# Split dataset

In [15]:
# Split dataset
test_data = my_data
test_indices = None  # will be converted to empty list in `split_dataset`, if also test_set_ratio == 0
val_data = my_data
val_indices = []
if config['test_pattern']:  # used if test data come from different files / file patterns
    test_data = data_class(config['data_dir'], pattern=config['test_pattern'], n_proc=-1, config=config)
    test_indices = test_data.all_IDs
if config['test_from']:  # load test IDs directly from file, if available, otherwise use `test_set_ratio`. Can work together with `test_pattern`
    test_indices = list(set([line.rstrip() for line in open(config['test_from']).readlines()]))
    try:
        test_indices = [int(ind) for ind in test_indices]  # integer indices
    except ValueError:
        pass  # in case indices are non-integers
    logger.info("Loaded {} test IDs from file: '{}'".format(len(test_indices), config['test_from']))
if config['val_pattern']:  # used if val data come from different files / file patterns
    val_data = data_class(config['data_dir'], pattern=config['val_pattern'], n_proc=-1, config=config)
    val_indices = val_data.all_IDs

In [16]:
# Note: currently a validation set must exist, either with `val_pattern` or `val_ratio`
# Using a `val_pattern` means that `val_ratio` == 0 and `test_ratio` == 0
if config['val_ratio'] > 0:
    train_indices, val_indices, test_indices = split_dataset(data_indices=my_data.all_IDs,
                                                             validation_method=validation_method,
                                                             n_splits=1,
                                                             validation_ratio=config['val_ratio'],
                                                             test_set_ratio=config['test_ratio'],  # used only if test_indices not explicitly specified
                                                             test_indices=test_indices,
                                                             random_seed=1337,
                                                             labels=labels)
    train_indices = train_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
    val_indices = val_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
else:
    train_indices = my_data.all_IDs
    if test_indices is None:
        test_indices = []

In [17]:
logger.info("{} samples may be used for training".format(len(train_indices)))
logger.info("{} samples will be used for validation".format(len(val_indices)))
logger.info("{} samples will be used for testing".format(len(test_indices)))

2023-05-10 17:07:53,011 | INFO : 4038 samples may be used for training
2023-05-10 17:07:53,012 | INFO : 1010 samples will be used for validation
2023-05-10 17:07:53,013 | INFO : 0 samples will be used for testing


In [18]:
with open(os.path.join(config['output_dir'], 'data_indices.json'), 'w') as f:
    try:
        json.dump({'train_indices': list(map(int, train_indices)),
                   'val_indices': list(map(int, val_indices)),
                   'test_indices': list(map(int, test_indices))}, f, indent=4)
    except ValueError:  # in case indices are non-integers
        json.dump({'train_indices': list(train_indices),
                   'val_indices': list(val_indices),
                   'test_indices': list(test_indices)}, f, indent=4)


In [19]:
# Pre-process features
normalizer = None
if config['norm_from']:
    with open(config['norm_from'], 'rb') as f:
        norm_dict = pickle.load(f)
    normalizer = Normalizer(**norm_dict)
elif config['normalization'] is not None:
    normalizer = Normalizer(config['normalization'])
    my_data.feature_df.loc[train_indices] = normalizer.normalize(my_data.feature_df.loc[train_indices])
    if not config['normalization'].startswith('per_sample'):
        # get normalizing values from training set and store for future use
        norm_dict = normalizer.__dict__
        with open(os.path.join(config['output_dir'], 'normalization.pickle'), 'wb') as f:
            pickle.dump(norm_dict, f, pickle.HIGHEST_PROTOCOL)
if normalizer is not None:
    if len(val_indices):
        val_data.feature_df.loc[val_indices] = normalizer.normalize(val_data.feature_df.loc[val_indices])
    if len(test_indices):
        test_data.feature_df.loc[test_indices] = normalizer.normalize(test_data.feature_df.loc[test_indices])


# Create model

In [20]:
# Create model
logger.info("Creating model ...")
model = model_factory(config, my_data)

if config['freeze']:
    for name, param in model.named_parameters():
        if name.startswith('output_layer'):
            param.requires_grad = True
        else:
            param.requires_grad = False

logger.info("Model:\n{}".format(model))
logger.info("Total number of parameters: {}".format(utils.count_parameters(model)))
logger.info("Trainable parameters: {}".format(utils.count_parameters(model, trainable=True)))


2023-05-10 17:07:53,108 | INFO : Creating model ...
2023-05-10 17:07:53,133 | INFO : Model:
TSTransformerEncoderClassiregressor(
  (project_inp): Linear(in_features=9, out_features=64, bias=True)
  (pos_enc): LearnablePositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerBatchNormEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (drop

# Initialize optimizer

In [21]:
# Initialize optimizer
if config['global_reg']:
    weight_decay = config['l2_reg']
    output_reg = None
else:
    weight_decay = 0
    output_reg = config['l2_reg']

optim_class = get_optimizer(config['optimizer'])
optimizer = optim_class(model.parameters(), lr=config['lr'], weight_decay=weight_decay)

start_epoch = 0
lr_step = 0  # current step index of `lr_step`
lr = config['lr']  # current learning step
# Load model and optimizer state
if args.load_model:
    model, optimizer, start_epoch = utils.load_model(model, config['load_model'], optimizer, config['resume'],
                                                     config['change_output'],
                                                     config['lr'],
                                                     config['lr_step'],
                                                     config['lr_factor'])
model.to(device)

loss_module = get_loss_module(config)

In [22]:
if config['test_only'] == 'testset':  # Only evaluate and skip training
    dataset_class, collate_fn, runner_class = pipeline_factory(config)
    test_dataset = dataset_class(test_data, test_indices)

    test_loader = DataLoader(dataset=test_dataset,
                             batch_size=config['batch_size'],
                             shuffle=False,
                             num_workers=config['num_workers'],
                             pin_memory=True,
                             collate_fn=lambda x: collate_fn(x, max_len=model.max_len))
    test_evaluator = runner_class(model, test_loader, device, loss_module,
                                        print_interval=config['print_interval'], console=config['console'])
    aggr_metrics_test, per_batch_test = test_evaluator.evaluate(keep_all=True)
    print_str = 'Test Summary: '
    for k, v in aggr_metrics_test.items():
        print_str += '{}: {:8f} | '.format(k, v)
    logger.info(print_str)
    #return

In [23]:
config['test_only'] 

In [24]:
# Initialize data generators
if config['test_only'] != 'testset':  # Only evaluate and skip training
    dataset_class, collate_fn, runner_class = pipeline_factory(config)
    val_dataset = dataset_class(val_data, val_indices)

    val_loader = DataLoader(dataset=val_dataset,
                            batch_size=config['batch_size'],
                            shuffle=False,
                            num_workers=config['num_workers'],
                            pin_memory=True,
                            collate_fn=lambda x: collate_fn(x, max_len=model.max_len))

    train_dataset = dataset_class(my_data, train_indices)

    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=config['batch_size'],
                              shuffle=True,
                              num_workers=config['num_workers'],
                              pin_memory=True,
                              collate_fn=lambda x: collate_fn(x, max_len=model.max_len))

    trainer = runner_class(model, train_loader, device, loss_module, optimizer, l2_reg=output_reg,
                                 print_interval=config['print_interval'], console=config['console'])
    val_evaluator = runner_class(model, val_loader, device, loss_module,
                                       print_interval=config['print_interval'], console=config['console'])

    tensorboard_writer = SummaryWriter(config['tensorboard_dir'])

    best_value = 1e16 if config['key_metric'] in NEG_METRICS else -1e16  # initialize with +inf or -inf depending on key metric
    metrics = []  # (for validation) list of lists: for each epoch, stores metrics like loss, ...
    best_metrics = {}

[2099 1225 2586 ... 4988  289 2292]
[3955  933 2495 ...  860  189 3223]


In [31]:
print(config["batch_size"])
for batch in train_loader:
    X, targets, padding_masks, IDs = batch
    print(X.shape)
    print(X[0:2])
    print(X.dtype)
    print("-"*20)
    print(targets.shape)
    print(targets[0])
    print(targets.dtype)
    print("-"*20)
    print(padding_masks.shape)
    print(padding_masks[0])
    print("-"*20)
    print(IDs)
    break

64
torch.Size([64, 24, 9])
tensor([[[-6.1151e-01, -2.2536e-01, -2.4848e-01, -9.8053e-01, -5.7372e-02,
           1.8004e-01,  6.8928e-01, -7.8279e-02, -5.2800e-01],
         [-6.1151e-01, -2.8206e-01, -2.4848e-01, -9.8053e-01, -5.7372e-02,
           2.1704e-01,  6.8928e-01, -7.8279e-02, -6.8756e-01],
         [-6.1151e-01, -2.8206e-01, -2.4848e-01, -9.8053e-01, -4.9087e-02,
           2.3555e-01,  6.9620e-01, -7.8279e-02, -3.6844e-01],
         [-6.1151e-01, -4.2382e-01, -3.2692e-01, -9.8053e-01, -3.2517e-02,
           2.8180e-01,  7.1004e-01, -7.8279e-02, -6.0778e-01],
         [-6.1151e-01, -4.5217e-01, -3.2692e-01, -9.8053e-01,  8.9086e-03,
           3.1880e-01,  6.9620e-01, -7.8279e-02, -2.8867e-01],
         [-6.1151e-01, -5.3722e-01, -3.2692e-01, -9.8053e-01,  3.3764e-02,
           3.7431e-01,  6.9620e-01, -7.8279e-02, -4.9328e-02],
         [-6.1151e-01, -5.3722e-01, -3.2692e-01, -9.8053e-01,  5.0334e-02,
           4.7607e-01,  5.7164e-01, -7.8279e-02,  3.4957e-01],
       

# Evaluate on validation before training

In [26]:
aggr_metrics_val, best_metrics, best_value = validate(val_evaluator, tensorboard_writer, config, best_metrics,
                                                      best_value, epoch=0)
metrics_names, metrics_values = zip(*aggr_metrics_val.items())
metrics.append(list(metrics_values))

2023-05-10 17:07:53,942 | INFO : Evaluating on validation set ...
2023-05-10 17:07:54,733 | INFO : Validation runtime: 0.0 hours, 0.0 minutes, 0.7899467945098877 seconds

2023-05-10 17:07:54,734 | INFO : Avg val. time: 0.0 hours, 0.0 minutes, 0.7899467945098877 seconds
2023-05-10 17:07:54,734 | INFO : Avg batch val. time: 0.04937167465686798 seconds
2023-05-10 17:07:54,734 | INFO : Avg sample val. time: 0.0007821255391187007 seconds
2023-05-10 17:07:54,735 | INFO : Epoch 0 Validation Summary: epoch: 0.000000 | loss: 32402.150743 | 


Evaluating Epoch 0   0.0% | batch:         0 of        16	|	loss: 26998.7
Evaluating Epoch 0   6.2% | batch:         1 of        16	|	loss: 31823.7
Evaluating Epoch 0  12.5% | batch:         2 of        16	|	loss: 33461.8
Evaluating Epoch 0  18.8% | batch:         3 of        16	|	loss: 27679.5
Evaluating Epoch 0  25.0% | batch:         4 of        16	|	loss: 19341
Evaluating Epoch 0  31.2% | batch:         5 of        16	|	loss: 35420
Evaluating Epoch 0  37.5% | batch:         6 of        16	|	loss: 23197.1
Evaluating Epoch 0  43.8% | batch:         7 of        16	|	loss: 43058.1
Evaluating Epoch 0  50.0% | batch:         8 of        16	|	loss: 34621.2
Evaluating Epoch 0  56.2% | batch:         9 of        16	|	loss: 53648
Evaluating Epoch 0  62.5% | batch:        10 of        16	|	loss: 27692.5
Evaluating Epoch 0  68.8% | batch:        11 of        16	|	loss: 47978.3
Evaluating Epoch 0  75.0% | batch:        12 of        16	|	loss: 20585.4
Evaluating Epoch 0  81.2% | batch:        13

/home/tianyi/anaconda3/envs/transformer/lib/python3.8/site-packages/numpy/lib/npyio.py:713: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


# Starting training...

In [27]:
logger.info('Starting training...')
for epoch in tqdm(range(start_epoch + 1, config["epochs"] + 1), desc='Training Epoch', leave=False):
    mark = epoch if config['save_all'] else 'last'
    epoch_start_time = time.time()
    aggr_metrics_train = trainer.train_epoch(epoch)  # dictionary of aggregate epoch metrics
    epoch_runtime = time.time() - epoch_start_time
    print()
    print_str = 'Epoch {} Training Summary: '.format(epoch)
    for k, v in aggr_metrics_train.items():
        tensorboard_writer.add_scalar('{}/train'.format(k), v, epoch)
        print_str += '{}: {:8f} | '.format(k, v)
    logger.info(print_str)
    logger.info("Epoch runtime: {} hours, {} minutes, {} seconds\n".format(*utils.readable_time(epoch_runtime)))
    total_epoch_time += epoch_runtime
    avg_epoch_time = total_epoch_time / (epoch - start_epoch)
    avg_batch_time = avg_epoch_time / len(train_loader)
    avg_sample_time = avg_epoch_time / len(train_dataset)
    logger.info("Avg epoch train. time: {} hours, {} minutes, {} seconds".format(*utils.readable_time(avg_epoch_time)))
    logger.info("Avg batch train. time: {} seconds".format(avg_batch_time))
    logger.info("Avg sample train. time: {} seconds".format(avg_sample_time))

    # evaluate if first or last epoch or at specified interval
    if (epoch == config["epochs"]) or (epoch == start_epoch + 1) or (epoch % config['val_interval'] == 0):
        aggr_metrics_val, best_metrics, best_value = validate(val_evaluator, tensorboard_writer, config,
                                                              best_metrics, best_value, epoch)
        metrics_names, metrics_values = zip(*aggr_metrics_val.items())
        metrics.append(list(metrics_values))

    utils.save_model(os.path.join(config['save_dir'], 'model_{}.pth'.format(mark)), epoch, model, optimizer)

    # Learning rate scheduling
    if epoch == config['lr_step'][lr_step]:
        utils.save_model(os.path.join(config['save_dir'], 'model_{}.pth'.format(epoch)), epoch, model, optimizer)
        lr = lr * config['lr_factor'][lr_step]
        if lr_step < len(config['lr_step']) - 1:  # so that this index does not get out of bounds
            lr_step += 1
        logger.info('Learning rate updated to: ', lr)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    # Difficulty scheduling
    if config['harden'] and check_progress(epoch):
        train_loader.dataset.update()
        val_loader.dataset.update()


2023-05-10 17:07:54,749 | INFO : Starting training...


Training Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/home/tianyi/Documents/mvts_transformer/src/optimizers.py:69: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at /opt/conda/conda-bld/pytorch_1670525541702/work/torch/csrc/utils/python_arg_parser.cpp:1420.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


Training Epoch 1   0.0% | batch:         0 of        64	|	loss: 50113.9
Training Epoch 1   1.6% | batch:         1 of        64	|	loss: 30074
Training Epoch 1   3.1% | batch:         2 of        64	|	loss: 32629.7
Training Epoch 1   4.7% | batch:         3 of        64	|	loss: 36068.2
Training Epoch 1   6.2% | batch:         4 of        64	|	loss: 24228.6
Training Epoch 1   7.8% | batch:         5 of        64	|	loss: 22839
Training Epoch 1   9.4% | batch:         6 of        64	|	loss: 22941
Training Epoch 1  10.9% | batch:         7 of        64	|	loss: 41369
Training Epoch 1  12.5% | batch:         8 of        64	|	loss: 34386.2
Training Epoch 1  14.1% | batch:         9 of        64	|	loss: 33905.6
Training Epoch 1  15.6% | batch:        10 of        64	|	loss: 29075.7
Training Epoch 1  17.2% | batch:        11 of        64	|	loss: 31694.9
Training Epoch 1  18.8% | batch:        12 of        64	|	loss: 44520.7
Training Epoch 1  20.3% | batch:        13 of        64	|	loss: 39493.6


2023-05-10 17:07:56,001 | INFO : Epoch 1 Training Summary: epoch: 1.000000 | loss: 30804.734507 | 
2023-05-10 17:07:56,002 | INFO : Epoch runtime: 0.0 hours, 0.0 minutes, 1.2432506084442139 seconds

2023-05-10 17:07:56,002 | INFO : Avg epoch train. time: 0.0 hours, 0.0 minutes, 1.2432506084442139 seconds
2023-05-10 17:07:56,003 | INFO : Avg batch train. time: 0.01942579075694084 seconds
2023-05-10 17:07:56,003 | INFO : Avg sample train. time: 0.00030788771878261856 seconds
2023-05-10 17:07:56,004 | INFO : Evaluating on validation set ...


...

Training Epoch 99  84.4% | batch:        54 of        64	|	loss: 2231.53
Training Epoch 99  85.9% | batch:        55 of        64	|	loss: 889.417
Training Epoch 99  87.5% | batch:        56 of        64	|	loss: 1155.94
Training Epoch 99  89.1% | batch:        57 of        64	|	loss: 937.909
Training Epoch 99  90.6% | batch:        58 of        64	|	loss: 945.103
Training Epoch 99  92.2% | batch:        59 of        64	|	loss: 1271.15
Training Epoch 99  93.8% | batch:        60 of        64	|	loss: 1618.8
Training Epoch 99  95.3% | batch:        61 of        64	|	loss: 1813.13
Training Epoch 99  96.9% | batch:        62 of        64	|	loss: 748.303
Training Epoch 99  98.4% | batch:        63 of        64	|	loss: 2071.85

Training Epoch 100   0.0% | batch:         0 of        64	|	loss: 599.459
Training Epoch 100   1.6% | batch:         1 of        64	|	loss: 1772.3
Training Epoch 100   3.1% | batch:         2 of        64	|	loss: 949.542
Training Epoch 100   4.7% | batch:         3 of  

2023-05-10 17:10:03,299 | INFO : Epoch 100 Training Summary: epoch: 100.000000 | loss: 1067.193284 | 
2023-05-10 17:10:03,300 | INFO : Epoch runtime: 0.0 hours, 0.0 minutes, 1.1849725246429443 seconds

2023-05-10 17:10:03,300 | INFO : Avg epoch train. time: 0.0 hours, 0.0 minutes, 1.195987823009491 seconds
2023-05-10 17:10:03,301 | INFO : Avg batch train. time: 0.018687309734523296 seconds
2023-05-10 17:10:03,301 | INFO : Avg sample train. time: 0.0002961832152078977 seconds
2023-05-10 17:10:03,302 | INFO : Evaluating on validation set ...


Training Epoch 100  87.5% | batch:        56 of        64	|	loss: 900.344
Training Epoch 100  89.1% | batch:        57 of        64	|	loss: 729.436
Training Epoch 100  90.6% | batch:        58 of        64	|	loss: 714.098
Training Epoch 100  92.2% | batch:        59 of        64	|	loss: 839.513
Training Epoch 100  93.8% | batch:        60 of        64	|	loss: 1202.13
Training Epoch 100  95.3% | batch:        61 of        64	|	loss: 768.368
Training Epoch 100  96.9% | batch:        62 of        64	|	loss: 1557.1
Training Epoch 100  98.4% | batch:        63 of        64	|	loss: 1444.34

Evaluating Epoch 100   0.0% | batch:         0 of        16	|	loss: 1583.32
Evaluating Epoch 100   6.2% | batch:         1 of        16	|	loss: 977.297
Evaluating Epoch 100  12.5% | batch:         2 of        16	|	loss: 1044.9
Evaluating Epoch 100  18.8% | batch:         3 of        16	|	loss: 1356.83
Evaluating Epoch 100  25.0% | batch:         4 of        16	|	loss: 1626.85
Evaluating Epoch 100  31.2% |

2023-05-10 17:10:03,450 | INFO : Validation runtime: 0.0 hours, 0.0 minutes, 0.14719152450561523 seconds

2023-05-10 17:10:03,451 | INFO : Avg val. time: 0.0 hours, 0.0 minutes, 0.1637059862797077 seconds
2023-05-10 17:10:03,451 | INFO : Avg batch val. time: 0.010231624142481731 seconds
2023-05-10 17:10:03,451 | INFO : Avg sample val. time: 0.00016208513493040367 seconds
2023-05-10 17:10:03,452 | INFO : Epoch 100 Validation Summary: epoch: 100.000000 | loss: 1923.427684 | 


Evaluating Epoch 100  43.8% | batch:         7 of        16	|	loss: 1942.74
Evaluating Epoch 100  50.0% | batch:         8 of        16	|	loss: 1783.44
Evaluating Epoch 100  56.2% | batch:         9 of        16	|	loss: 6761.94
Evaluating Epoch 100  62.5% | batch:        10 of        16	|	loss: 1492.08
Evaluating Epoch 100  68.8% | batch:        11 of        16	|	loss: 2025.48
Evaluating Epoch 100  75.0% | batch:        12 of        16	|	loss: 1512.86
Evaluating Epoch 100  81.2% | batch:        13 of        16	|	loss: 1370.69
Evaluating Epoch 100  87.5% | batch:        14 of        16	|	loss: 1378.71
Evaluating Epoch 100  93.8% | batch:        15 of        16	|	loss: 2168.78



In [28]:
# Export evolution of metrics over epochs
header = metrics_names
metrics_filepath = os.path.join(config["output_dir"], "metrics_" + config["experiment_name"] + ".xls")
book = utils.export_performance_metrics(metrics_filepath, metrics, header, sheet_name="metrics")

# Export record metrics to a file accumulating records from all experiments
utils.register_record(config["records_file"], config["initial_timestamp"], config["experiment_name"],
                      best_metrics, aggr_metrics_val, comment=config['comment'])

logger.info('Best {} was {}. Other metrics: {}'.format(config['key_metric'], best_value, best_metrics))
logger.info('All Done!')

total_runtime = time.time() - total_start_time
logger.info("Total runtime: {} hours, {} minutes, {} seconds\n".format(*utils.readable_time(total_runtime)))

#return best_value
print(best_value)

2023-05-10 17:10:03,472 | INFO : Exported per epoch performance metrics in '../experiments/pm25_fromScratch_Regression_2023-05-10_17-07-32_kDk/metrics_pm25_fromScratch_Regression.xls'
2023-05-10 17:10:03,475 | INFO : Exported performance record to 'Regression_records.xls'
2023-05-10 17:10:03,476 | INFO : Best loss was 1705.1961362933168. Other metrics: OrderedDict([('epoch', 48), ('loss', 1705.1961362933168)])
2023-05-10 17:10:03,476 | INFO : All Done!
2023-05-10 17:10:03,477 | INFO : Total runtime: 0.0 hours, 2.0 minutes, 10.478641271591187 seconds



1705.1961362933168


In [29]:

def main(config):

    total_epoch_time = 0
    total_eval_time = 0

    total_start_time = time.time()

    # Add file logging besides stdout
    file_handler = logging.FileHandler(os.path.join(config['output_dir'], 'output.log'))
    logger.addHandler(file_handler)

    logger.info('Running:\n{}\n'.format(' '.join(sys.argv)))  # command used to run

    if config['seed'] is not None:
        torch.manual_seed(config['seed'])

    device = torch.device('cuda' if (torch.cuda.is_available() and config['gpu'] != '-1') else 'cpu')
    logger.info("Using device: {}".format(device))
    if device == 'cuda':
        logger.info("Device index: {}".format(torch.cuda.current_device()))

    # Build data
    logger.info("Loading and preprocessing data ...")
    data_class = data_factory[config['data_class']]
    my_data = data_class(config['data_dir'], pattern=config['pattern'], n_proc=config['n_proc'], limit_size=config['limit_size'], config=config)
    feat_dim = my_data.feature_df.shape[1]  # dimensionality of data features
    if config['task'] == 'classification':
        validation_method = 'StratifiedShuffleSplit'
        labels = my_data.labels_df.values.flatten()
    else:
        validation_method = 'ShuffleSplit'
        labels = None

    # Split dataset
    test_data = my_data
    test_indices = None  # will be converted to empty list in `split_dataset`, if also test_set_ratio == 0
    val_data = my_data
    val_indices = []
    if config['test_pattern']:  # used if test data come from different files / file patterns
        test_data = data_class(config['data_dir'], pattern=config['test_pattern'], n_proc=-1, config=config)
        test_indices = test_data.all_IDs
    if config['test_from']:  # load test IDs directly from file, if available, otherwise use `test_set_ratio`. Can work together with `test_pattern`
        test_indices = list(set([line.rstrip() for line in open(config['test_from']).readlines()]))
        try:
            test_indices = [int(ind) for ind in test_indices]  # integer indices
        except ValueError:
            pass  # in case indices are non-integers
        logger.info("Loaded {} test IDs from file: '{}'".format(len(test_indices), config['test_from']))
    if config['val_pattern']:  # used if val data come from different files / file patterns
        val_data = data_class(config['data_dir'], pattern=config['val_pattern'], n_proc=-1, config=config)
        val_indices = val_data.all_IDs

    # Note: currently a validation set must exist, either with `val_pattern` or `val_ratio`
    # Using a `val_pattern` means that `val_ratio` == 0 and `test_ratio` == 0
    if config['val_ratio'] > 0:
        train_indices, val_indices, test_indices = split_dataset(data_indices=my_data.all_IDs,
                                                                 validation_method=validation_method,
                                                                 n_splits=1,
                                                                 validation_ratio=config['val_ratio'],
                                                                 test_set_ratio=config['test_ratio'],  # used only if test_indices not explicitly specified
                                                                 test_indices=test_indices,
                                                                 random_seed=1337,
                                                                 labels=labels)
        train_indices = train_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
        val_indices = val_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
    else:
        train_indices = my_data.all_IDs
        if test_indices is None:
            test_indices = []

    logger.info("{} samples may be used for training".format(len(train_indices)))
    logger.info("{} samples will be used for validation".format(len(val_indices)))
    logger.info("{} samples will be used for testing".format(len(test_indices)))

    with open(os.path.join(config['output_dir'], 'data_indices.json'), 'w') as f:
        try:
            json.dump({'train_indices': list(map(int, train_indices)),
                       'val_indices': list(map(int, val_indices)),
                       'test_indices': list(map(int, test_indices))}, f, indent=4)
        except ValueError:  # in case indices are non-integers
            json.dump({'train_indices': list(train_indices),
                       'val_indices': list(val_indices),
                       'test_indices': list(test_indices)}, f, indent=4)

    # Pre-process features
    normalizer = None
    if config['norm_from']:
        with open(config['norm_from'], 'rb') as f:
            norm_dict = pickle.load(f)
        normalizer = Normalizer(**norm_dict)
    elif config['normalization'] is not None:
        normalizer = Normalizer(config['normalization'])
        my_data.feature_df.loc[train_indices] = normalizer.normalize(my_data.feature_df.loc[train_indices])
        if not config['normalization'].startswith('per_sample'):
            # get normalizing values from training set and store for future use
            norm_dict = normalizer.__dict__
            with open(os.path.join(config['output_dir'], 'normalization.pickle'), 'wb') as f:
                pickle.dump(norm_dict, f, pickle.HIGHEST_PROTOCOL)
    if normalizer is not None:
        if len(val_indices):
            val_data.feature_df.loc[val_indices] = normalizer.normalize(val_data.feature_df.loc[val_indices])
        if len(test_indices):
            test_data.feature_df.loc[test_indices] = normalizer.normalize(test_data.feature_df.loc[test_indices])

    # Create model
    logger.info("Creating model ...")
    model = model_factory(config, my_data)

    if config['freeze']:
        for name, param in model.named_parameters():
            if name.startswith('output_layer'):
                param.requires_grad = True
            else:
                param.requires_grad = False

    logger.info("Model:\n{}".format(model))
    logger.info("Total number of parameters: {}".format(utils.count_parameters(model)))
    logger.info("Trainable parameters: {}".format(utils.count_parameters(model, trainable=True)))


    # Initialize optimizer

    if config['global_reg']:
        weight_decay = config['l2_reg']
        output_reg = None
    else:
        weight_decay = 0
        output_reg = config['l2_reg']

    optim_class = get_optimizer(config['optimizer'])
    optimizer = optim_class(model.parameters(), lr=config['lr'], weight_decay=weight_decay)

    start_epoch = 0
    lr_step = 0  # current step index of `lr_step`
    lr = config['lr']  # current learning step
    # Load model and optimizer state
    if args.load_model:
        model, optimizer, start_epoch = utils.load_model(model, config['load_model'], optimizer, config['resume'],
                                                         config['change_output'],
                                                         config['lr'],
                                                         config['lr_step'],
                                                         config['lr_factor'])
    model.to(device)

    loss_module = get_loss_module(config)

    if config['test_only'] == 'testset':  # Only evaluate and skip training
        dataset_class, collate_fn, runner_class = pipeline_factory(config)
        test_dataset = dataset_class(test_data, test_indices)

        test_loader = DataLoader(dataset=test_dataset,
                                 batch_size=config['batch_size'],
                                 shuffle=False,
                                 num_workers=config['num_workers'],
                                 pin_memory=True,
                                 collate_fn=lambda x: collate_fn(x, max_len=model.max_len))
        test_evaluator = runner_class(model, test_loader, device, loss_module,
                                            print_interval=config['print_interval'], console=config['console'])
        aggr_metrics_test, per_batch_test = test_evaluator.evaluate(keep_all=True)
        print_str = 'Test Summary: '
        for k, v in aggr_metrics_test.items():
            print_str += '{}: {:8f} | '.format(k, v)
        logger.info(print_str)
        return
    
    # Initialize data generators
    dataset_class, collate_fn, runner_class = pipeline_factory(config)
    val_dataset = dataset_class(val_data, val_indices)

    val_loader = DataLoader(dataset=val_dataset,
                            batch_size=config['batch_size'],
                            shuffle=False,
                            num_workers=config['num_workers'],
                            pin_memory=True,
                            collate_fn=lambda x: collate_fn(x, max_len=model.max_len))

    train_dataset = dataset_class(my_data, train_indices)

    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=config['batch_size'],
                              shuffle=True,
                              num_workers=config['num_workers'],
                              pin_memory=True,
                              collate_fn=lambda x: collate_fn(x, max_len=model.max_len))

    trainer = runner_class(model, train_loader, device, loss_module, optimizer, l2_reg=output_reg,
                                 print_interval=config['print_interval'], console=config['console'])
    val_evaluator = runner_class(model, val_loader, device, loss_module,
                                       print_interval=config['print_interval'], console=config['console'])

    tensorboard_writer = SummaryWriter(config['tensorboard_dir'])

    best_value = 1e16 if config['key_metric'] in NEG_METRICS else -1e16  # initialize with +inf or -inf depending on key metric
    metrics = []  # (for validation) list of lists: for each epoch, stores metrics like loss, ...
    best_metrics = {}

    # Evaluate on validation before training
    aggr_metrics_val, best_metrics, best_value = validate(val_evaluator, tensorboard_writer, config, best_metrics,
                                                          best_value, epoch=0)
    metrics_names, metrics_values = zip(*aggr_metrics_val.items())
    metrics.append(list(metrics_values))

    logger.info('Starting training...')
    for epoch in tqdm(range(start_epoch + 1, config["epochs"] + 1), desc='Training Epoch', leave=False):
        mark = epoch if config['save_all'] else 'last'
        epoch_start_time = time.time()
        aggr_metrics_train = trainer.train_epoch(epoch)  # dictionary of aggregate epoch metrics
        epoch_runtime = time.time() - epoch_start_time
        print()
        print_str = 'Epoch {} Training Summary: '.format(epoch)
        for k, v in aggr_metrics_train.items():
            tensorboard_writer.add_scalar('{}/train'.format(k), v, epoch)
            print_str += '{}: {:8f} | '.format(k, v)
        logger.info(print_str)
        logger.info("Epoch runtime: {} hours, {} minutes, {} seconds\n".format(*utils.readable_time(epoch_runtime)))
        total_epoch_time += epoch_runtime
        avg_epoch_time = total_epoch_time / (epoch - start_epoch)
        avg_batch_time = avg_epoch_time / len(train_loader)
        avg_sample_time = avg_epoch_time / len(train_dataset)
        logger.info("Avg epoch train. time: {} hours, {} minutes, {} seconds".format(*utils.readable_time(avg_epoch_time)))
        logger.info("Avg batch train. time: {} seconds".format(avg_batch_time))
        logger.info("Avg sample train. time: {} seconds".format(avg_sample_time))

        # evaluate if first or last epoch or at specified interval
        if (epoch == config["epochs"]) or (epoch == start_epoch + 1) or (epoch % config['val_interval'] == 0):
            aggr_metrics_val, best_metrics, best_value = validate(val_evaluator, tensorboard_writer, config,
                                                                  best_metrics, best_value, epoch)
            metrics_names, metrics_values = zip(*aggr_metrics_val.items())
            metrics.append(list(metrics_values))

        utils.save_model(os.path.join(config['save_dir'], 'model_{}.pth'.format(mark)), epoch, model, optimizer)

        # Learning rate scheduling
        if epoch == config['lr_step'][lr_step]:
            utils.save_model(os.path.join(config['save_dir'], 'model_{}.pth'.format(epoch)), epoch, model, optimizer)
            lr = lr * config['lr_factor'][lr_step]
            if lr_step < len(config['lr_step']) - 1:  # so that this index does not get out of bounds
                lr_step += 1
            logger.info('Learning rate updated to: ', lr)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

        # Difficulty scheduling
        if config['harden'] and check_progress(epoch):
            train_loader.dataset.update()
            val_loader.dataset.update()

    # Export evolution of metrics over epochs
    header = metrics_names
    metrics_filepath = os.path.join(config["output_dir"], "metrics_" + config["experiment_name"] + ".xls")
    book = utils.export_performance_metrics(metrics_filepath, metrics, header, sheet_name="metrics")

    # Export record metrics to a file accumulating records from all experiments
    utils.register_record(config["records_file"], config["initial_timestamp"], config["experiment_name"],
                          best_metrics, aggr_metrics_val, comment=config['comment'])

    logger.info('Best {} was {}. Other metrics: {}'.format(config['key_metric'], best_value, best_metrics))
    logger.info('All Done!')

    total_runtime = time.time() - total_start_time
    logger.info("Total runtime: {} hours, {} minutes, {} seconds\n".format(*utils.readable_time(total_runtime)))

    return best_value


In [30]:
5*10**5

500000